<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [21]:
from dotmap import DotMap
import pandas as pd

import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range, todf
from tools.trust_ncg import minimize
import tools.che as che
R=8.314 # J/(mol K)

In [22]:

p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c):
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    W_n_vap = Pw * V_vap / (R * c.T)
    P = (c.air_n + W_n_vap) * R * c.T / V_vap
    return (s.P_f - P)**2

s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(360,300,600)


vsc=VSC(v,s)
model_f = vsc.transform(model)
model_f(vsc.x)

DeviceArray(2.89480362e+08, dtype=float64)

In [23]:
modelf = jax.jit(vsc.transform(model))
x_min, f = minimize(modelf, vsc.x, verbosity=0)
vsc.xtov(x_min)

DotMap(T=DeviceArray([365.71923532], dtype=float64), _ipython_display_=DotMap(), _repr_mimebundle_=DotMap())

In [26]:
def model2(c):
    r=DotMap()
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    r.W_n_vap = Pw * V_vap / (R * c.T)
    P = (s.air_n + r.W_n_vap) * R * c.T / V_vap
    return ((s.P_f - P)/(s.P_f+P))**2 + ((r.W_n_vap - 0.3)/(r.W_n_vap + 0.3))**2, r

In [27]:
s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, 300, 400)
v.V_tot = Range(0.015, 0., 0.03)

vsc=VSC(v,s)

In [28]:
modelf = jax.jit(vsc.transform(model2))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

0.10764694306583555
0.006885656391338845
0.006885656391338845
0.000567926449394903
0.000567926449394903
9.915253855269998e-05
9.938149952270923e-08
9.938149952270923e-08
9.938149952270923e-08
3.6759278704897935e-11
3.6759278704897935e-11
3.6759278704897935e-11
3.6759278704897935e-11
2.2035016056715132e-15


In [29]:
vsc.generate_reports(model2, x_min)

In [30]:
vsc.rdf

,vector1
,1
W_n_vap,0.3


In [31]:
vsc.sdf

,vector1
,1
W_tot,1.000000
V_tot,0.010000
P_i,100000.000000
P_f,200000.000000
T_i,298.000000
Vw_i,0.001005
Vair_i,0.008995
air_n,0.363057


In [32]:
vsc.vdf

,vector1
,1
T,370.005636
V_tot,0.011204


In [33]:
def model3(c):
    r=DotMap()
    Pw = p.Pvap(c.T)[3]
    r.v_w_vap = c.W_n_vap*18e-3/1e3
    V_vap = c.V_tot - (c.Vw_i - r.v_w_vap)
    r.W_n_vap = Pw * V_vap / (R * c.T)
    P = (s.air_n + r.W_n_vap) * R * c.T / V_vap
    return ((s.P_f - P)/(s.P_f + P))**2 + ((c.W_n_vap - r.W_n_vap)/(c.W_n_vap + r.W_n_vap))**2, r

In [34]:
s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350., 300., 400.)
v.W_n_vap = Range(0.5, 0., 1.)

vsc=VSC(v,s)

In [35]:
modelf = vsc.transform(model3)
modelf(vsc.x)

DeviceArray(0.36084172, dtype=float64)

In [36]:
x_min, f = minimize(modelf, vsc.x, verbosity=1)

0.36084172408313475
0.08354712376249757
0.08354712376249757
0.08354712376249757
0.07485437302177236
0.01067352149307918
0.01067352149307918
0.001569833199205387
0.001569833199205387
9.26531893088015e-06
9.26531893088015e-06
4.7044007572671604e-06
4.7044007572671604e-06
1.0589344003515588e-10
1.0589344003515588e-10
1.0589344003515588e-10
4.7682387238662576e-11
4.7682387238662576e-11
4.7682387238662576e-11
1.3760836806900143e-16


In [37]:
vsc.generate_reports(model3,x_min)

In [38]:
vsc.rdf

,vector1
,1
v_w_vap,0.000004
W_n_vap,0.228847


In [39]:
vsc.vdf

,vector1
,1
T,365.736681
W_n_vap,0.228847


In [40]:
vsc.sdf

,vector1
,1
W_tot,1.000000
V_tot,0.010000
P_i,100000.000000
P_f,200000.000000
T_i,298.000000
Vw_i,0.001005
Vair_i,0.008995
air_n,0.363057
